# SETUP

In [ ]:
/*-- Create the customer_support_emails table
CREATE OR REPLACE TABLE customer_support_emails (
    case_number STRING,                 -- Unique case number for tracking
    email_id STRING PRIMARY KEY,        -- Unique ID for each email
    email_from STRING,                  -- Sender's email address
    email_to STRING,                    -- Recipient's email address (support team address)
    email_cc STRING,                    -- Any CC recipients, if applicable
    email_bcc STRING,                   -- Any BCC recipients, if applicable
    subject STRING,                     -- Subject line of the email
    email_body STRING,                  -- Full body of the email
    received_time TIMESTAMP_NTZ,        -- Timestamp of when the email was received
    email_status STRING,                -- Status (e.g., 'Open', 'In Progress', 'Closed')
    priority STRING,                    -- Priority of the email (e.g., 'High', 'Medium', 'Low')
    response_due_by TIMESTAMP_NTZ,      -- Deadline for a response, if applicable
    assigned_to STRING,                 -- Support agent or team assigned to the email
    tags ARRAY,                         -- Array of tags or keywords for categorization
    attachments ARRAY,                  -- Array containing metadata about attachments (if applicable)
    cortex_sentiment STRING,            -- Sentiment of the email (left blank initially)
    cortex_sentiment_normalized STRING, -- For ranking Sentiment 1-10
    cortex_summarize STRING,            -- Category for summarization tasks
    cortex_classify_text STRING,        -- Category for text classification
    cortex_extract_answer STRING,       -- Category for answer extraction
    cortex_translate STRING,            -- Category for translation
    cortex_detected_language STRING    -- What language was detected
);


In [ ]:
/*ALTER TABLE customer_support_emails
ADD COLUMN case_number STRING; 

In [ ]:
/*SELECT COUNT(*) AS count_missing_sentiment
FROM CUSTOMER_SUPPORT_DB.PUBLIC.CUSTOMER_SUPPORT_EMAILS
WHERE cortex_sentiment_normalized IS NULL 
   OR TRIM(cortex_sentiment_normalized) = '';

In [ ]:
/*SELECT *
FROM CUSTOMER_SUPPORT_EMAILS
WHERE EMAIL_BODY IS NOT NULL
AND (
    (CORTEX_TRANSLATE IS NULL OR CORTEX_TRANSLATE = '') AND (CORTEX_DETECTED_LANGUAGE IS NULL OR CORTEX_DETECTED_LANGUAGE = '')
    OR CORTEX_SENTIMENT IS NULL OR CORTEX_SENTIMENT = ''
    OR (CORTEX_SUMMARIZE IS NULL OR CORTEX_SUMMARIZE = '') AND LENGTH(EMAIL_BODY) > 5
)

In [ ]:
/*SELECT * 
FROM CUSTOMER_SUPPORT_EMAILS 
WHERE (CORTEX_CLASSIFY_TEXT IS NULL OR CORTEX_CLASSIFY_TEXT = '') 
AND LENGTH(EMAIL_BODY) > 5 
LIMIT 10

In [ ]:
/*-- Query to see results
SELECT 
    CORTEX_CLASSIFY_TEXT as Category,
    COUNT(*) as Count
FROM CUSTOMER_SUPPORT_EMAILS
GROUP BY CORTEX_CLASSIFY_TEXT
ORDER BY Count DESC;

In [ ]:
SELECT * FROM CUSTOMER_SUPPORT_DB.PUBLIC.CUSTOMER_SUPPORT_EMAILS;

In [ ]:
SELECT * FROM CUSTOMER_SUPPORT_DB.PUBLIC.CUSTOMER_SUPPORT_EMAILS where EMAIL_ID = '13a0fe99-0beb-4c80-b9cf-ff97a2a7bceb'

In [ ]:
# Import Python packages
import streamlit as st
import pandas as pd
import uuid
import random
from faker import Faker
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import lit, concat, random as snowflake_random, when, array_construct
from datetime import datetime, timedelta

# Initialize Faker
fake = Faker()

# Snowflake session
session = get_active_session()

# Generate 100 rows with UUIDs and other random values in a Pandas DataFrame
num_rows        = 25
subject_list    = []
body_list       = []
email_from_list = []

# Paired subjects and email bodies with Callaway product names
email_content = [
    ("Issue with Paradym Ai Smoke MAX driver durability", 
     "My Paradym Ai Smoke MAX driver isn’t holding up well. The grip is wearing out faster than expected."),
    ("Performance concerns with Chrome Soft golf ball", 
     "I’m not getting the expected distance with Chrome Soft golf balls. Could it be a product issue?"),
    ("Warranty question for Big Bertha REVA irons", 
     "I have warranty questions regarding my Big Bertha REVA irons, which have developed cracks."),
    ("Replacement request for defective Apex Pro iron set", 
     "My Apex Pro iron set arrived with scratches. I’d like to request a replacement."),
    ("Request for refund on Jaws Raw Face Chrome wedge", 
     "The Jaws Raw Face Chrome wedge is not performing as expected. Can I get a refund or exchange?"),
    ("Problem with Fairway 14 Stand Bag zipper", 
     "The zipper on my Fairway 14 Stand Bag is broken. Can it be repaired or replaced?"),
    ("Poor control with Paradym Ai Smoke irons", 
     "I’ve been using Paradym Ai Smoke irons, but they seem to lack control on the course. Any advice?"),
    ("Alignment issues with Jaws Raw Face Chrome wedge", 
     "I’m facing alignment issues with my Jaws Raw Face Chrome wedge. Could this be a defect?"),
    ("Callaway driver Paradym Ai Smoke not performing as expected", 
     "The Paradym Ai Smoke driver isn't delivering as promised. Can I get some advice on adjustments?"),
    ("Questions about Supersoft golf ball compression", 
     "I am disappointed with the Supersoft golf ball. It doesn’t meet my expectations for distance."),
    ("Broken shaft on my Paradym Ai Smoke Triple Diamond driver", 
     "My Paradym Ai Smoke Triple Diamond driver’s shaft broke after minimal use. Is this under warranty?"),
    ("Need guidance on loft adjustment for XR 13-Piece Set", 
     "The loft on my XR 13-Piece Set driver seems off. Could this be affecting my performance?"),
    ("Request for replacement grip on Paradym Ai Smoke MAX driver", 
     "The grip on my Paradym Ai Smoke MAX driver has worn down. How can I order a replacement?"),
    ("Complaint about Fairway C Stand Bag durability", 
     "The Fairway C Stand Bag has torn after a few uses. I expected better durability from Callaway."),
    ("Apex Pro irons showing wear too quickly", 
     "I’m noticing wear on my Apex Pro irons after limited play. Is this normal?"),
    ("Battery issue with Callaway GPS device", 
     "My Callaway GPS device battery isn’t holding a charge. Can I get a replacement?"),
    ("My Fairway 14 Stand Bag has broken straps", 
     "The straps on my Fairway 14 Stand Bag broke during use. Is there a way to repair it?"),
    ("Question about customizing Paradym Ai Smoke driver settings", 
     "I’d like to customize my Paradym Ai Smoke driver settings. Can you provide some guidance?"),
    ("Inconsistent performance of ERC Soft golf balls", 
     "The ERC Soft golf balls feel inconsistent in spin control. Could this be a batch issue?"),
    ("Disappointed with Callaway Chrome Tour golf ball results", 
     "I bought the Chrome Tour golf ball but haven’t seen the promised performance gains. Can you advise?"),
    ("Experiencing issues with the durability of the Paradym Ai Smoke MAX driver, particularly with grip wear after limited use", 
     "The grip on my Paradym Ai Smoke MAX driver is wearing out quickly, and I am concerned about its overall durability."),
    ("Need assistance with warranty and replacement process for cracked Big Bertha REVA irons", 
     "My Big Bertha REVA irons have developed cracks, and I need guidance on the warranty and replacement process. I purchased these irons about six months ago, and I was under the impression that they would be covered by Callaway's warranty policy for any defects or issues that arose during normal use. I have been extremely careful with my equipment, and it is very disappointing to see these cracks appearing so soon. I would greatly appreciate it if you could provide me with detailed information on how to proceed with a warranty claim. Do I need to bring these irons to an authorized Callaway dealer, or can I initiate the process online? What documentation will I need to provide, and how long does the typical warranty process take? I am eager to get back on the course with a fully functional set of clubs, so any assistance you can provide would be very helpful."),
    
    # Callaway-specific options in different languages
    ("Solicitud de reemplazo para juego de hierros Apex Pro defectuoso", 
     "Mi juego de hierros Apex Pro llegó con rasguños. Me gustaría solicitar un reemplazo."),
    ("Problema con la cremallera de la bolsa Fairway 14 Stand", 
     "La cremallera de mi bolsa Fairway 14 Stand está rota. ¿Puede repararse o reemplazarse?"),
    ("Supersoft ゴルフボールの圧縮についての質問", 
     "Supersoft ゴルフボールが期待通りの距離を提供していません。製品の問題ですか？"),
    ("Consulta sobre configuración personalizada de Paradym Ai Smoke ドライバー", 
     "Paradym Ai Smoke ドライバーの設定をカスタマイズしたいのですが、アドバイスをいただけますか？"),
    ("Richiesta di assistenza per il driver Paradym Ai Smoke MAX", 
     "Il mio driver Paradym Ai Smoke MAX non sta funzionando come previsto. È possibile una sostituzione?"),
    ("Domanda sulla durata della batteria del dispositivo GPS Callaway", 
     "La batteria del mio dispositivo GPS Callaway non dura come previsto. Posso ottenere una sostituzione?"),
    ("Problema di durata della borsa Callaway Fairway 14", 
     "La borsa Callaway Fairway 14 si è strappata dopo pochi utilizzi. Mi aspettavo una qualità migliore."),
    ("Problème de durabilité du driver Paradym Ai Smoke MAX", 
     "Mon driver Paradym Ai Smoke MAX ne tient pas bien. La poignée s'use plus vite que prévu. Est-ce un défaut de fabrication ?"),
    ("Question sur la garantie des fers Big Bertha REVA", 
     "J'ai des questions sur la garantie concernant mes fers Big Bertha REVA, qui présentent des fissures. Pouvez-vous m'aider avec le processus de remplacement ?"),
    ("Проблема с молнией сумки Fairway 14 Stand", 
     "Молния на моей сумке Fairway 14 Stand сломана. Можно ли её отремонтировать или заменить?"),
    ("Вопрос о долговечности драйвера Paradym Ai Smoke MAX", 
     "Мой драйвер Paradym Ai Smoke MAX не выдерживает долгого использования, рукоятка изнашивается быстрее, чем ожидалось. Это покрывается гарантией?"),
    ("Problème de peinture sur le bois de parcours Rogue ST MAX", 
     "La peinture de mon bois de parcours Rogue ST MAX s'écaille après seulement quelques utilisations. Cela est-il couvert par la garantie ?"),
    ("Besoin d'aide pour ajuster le loft sur les fers Big Bertha B21", 
     "Je souhaite ajuster le loft de mes fers Big Bertha B21 pour améliorer la trajectoire. Quels sont les réglages recommandés et pouvez-vous me donner des conseils ?"),
    ("Frage zur Garantie für den Callaway Rogue ST MAX", 
     "Mein Callaway Rogue ST MAX Driver zeigt Risse nach wenigen Runden. Ist dies durch die Garantie abgedeckt? Wie reiche ich einen Garantieanspruch ein?"),
    ("Problema di ruggine con il wedge Jaws Full Toe", 
     "Il mio wedge Jaws Full Toe si sta arrugginendo molto rapidamente. È normale per questo modello o si tratta di un difetto?"),
    ("Consulta sobre la personalización del driver Epic Speed", 
     "Quisiera personalizar mi driver Epic Speed para optimizar el sesgo de draw o fade. ¿Es posible y qué opciones tengo?"),
    ("문의 사항: Callaway XR Speed 드라이버의 거리 문제", 
     "최근 Callaway XR Speed 드라이버로 거리가 줄어들고 있습니다. 클럽 페이스의 마모 때문일까요? 해결 방법이 있을까요?"),
    ("Pytanie o wymianę uszkodzonego uchwytu na driverze MAVRIK Max", 
     "Uchwyt w moim driverze MAVRIK Max jest uszkodzony. Czy mogę zamówić taki sam uchwyt od Callaway?"),
    ("Dúvida sobre desgaste prematuro do driver Big Bertha REVA", 
     "Meu driver Big Bertha REVA está apresentando desgaste prematuro na empunhadura. Isso é coberto pela garantia?"),
    ("Frage zu Anpassungsoptionen für den Callaway Epic Flash Sub Zero", 
     "Ich möchte meinen Callaway Epic Flash Sub Zero Driver anpassen, um die Leistung zu verbessern. Gibt es Empfehlungen für professionelle Anpassungen?"),
    ("Problème de fermeture éclair sur le sac Callaway ORG 14", 
     "La fermeture éclair de mon sac Callaway ORG 14 est cassée après seulement un mois d'utilisation. Puis-je le faire réparer ou remplacer sous garantie ?"),
    ("Problema con la cerniera della borsa Hyper Lite Zero", 
     "La cerniera della mia borsa Hyper Lite Zero si è rotta. Come posso ottenere una riparazione o una sostituzione tramite la garanzia di Callaway?"),
    ("Frage zur Lebensdauer der Odyssey Stroke Lab Putter", 
     "Mein Odyssey Stroke Lab Putter zeigt bereits nach kurzer Nutzung Verschleiß. Ist dies ein Herstellungsfehler?"),
    ("Problema z rdzewieniem wedża Jaws Full Toe", 
     "Mój wedge Jaws Full Toe rdzewieje znacznie szybciej niż się spodziewałem. Czy to normalne? Jak powinienem go konserwować?"),
    ("구입한 MAVRIK Max 하이브리드 맞춤 피팅 요청", 
     "MAVRIK Max 하이브리드를 맞춤 피팅하고 싶습니다. 어떤 조정이 가능하고 가까운 피터를 추천받을 수 있을까요?"),
    ("Consulta sobre el reemplazo de la cubierta del driver Rogue ST MAX", 
     "La cubierta de mi driver Rogue ST MAX está dañada. ¿Puedo solicitar un reemplazo?"),
    ("問題: Odyssey Triple Track パターのアライメントが薄れてきています", 
     "Odyssey Triple Track パターのアライメントが薄れてきて、ボールをラインアップするのが難しくなっています。交換や修復の方法はありますか？")
]

# Adding more specific Callaway product-related issues
email_content.extend([
    ("Rogue ST MAX Driver head cracked",
     "The head of my Rogue ST MAX Driver developed a crack after a few rounds. Is it covered by warranty? I have always been very cautious with my clubs, and I was quite surprised to see the crack appear. It has made the driver practically unusable, as I am worried that continued use might make the damage worse or even dangerous. I purchased the driver less than a year ago, and I believe it should still be under warranty. Could you please provide detailed steps on how to claim a warranty replacement or repair? Should I bring it to the retailer where I bought it, or can I send it directly to Callaway? What documents or proof of purchase do I need to provide? Any assistance in resolving this matter would be greatly appreciated."),
    ("Odyssey White Hot Putter alignment problem",
     "The alignment of my Odyssey White Hot Putter seems off. Could this be a manufacturing defect? I noticed that when I line up my shots, the putter head doesn't seem to align properly with the ball, which is causing my shots to veer off course. I have tried adjusting my grip and stance, but the issue persists. I have compared it with another putter, and the misalignment seems to be a fault with the club itself. I would like to know if this kind of issue is something that is covered under warranty, and if there is a way to get a replacement or repair. This putter is crucial to my short game, and I am eager to have it functioning correctly."),
    ("Epic Flash Fairway Wood losing distance",
     "I have noticed my Epic Flash Fairway Wood is losing distance. Any suggestions on what could be wrong? Over the past few weeks, I have found that I am not achieving the same yardage as I used to with my Epic Flash Fairway Wood. I have not made any significant changes to my swing, and I have tried using different golf balls, but the issue remains. Could this be related to the clubface or shaft? I would appreciate any advice you can offer on how to diagnose and fix this problem. Should I consider re-shafting the club, or could it be a problem with the clubface losing its 'pop'? Any help in restoring the performance of this club would be greatly valued."),
    ("Callaway MAVRIK Irons shaft flex issue",
     "The shaft on my MAVRIK Irons feels too flexible despite selecting the stiff option. Could it be mislabeled? When I ordered my MAVRIK Irons, I specifically chose the stiff shaft option to suit my swing speed. However, the clubs feel much more flexible than expected, and it is affecting my performance, especially in terms of accuracy. I am concerned that there might have been a mix-up during manufacturing or shipping. I would like to know if there is a way to verify the shaft specifications and, if necessary, get a replacement. I have been loyal to Callaway for many years, and I hope this issue can be resolved quickly so that I can get back to enjoying my game."),
    ("Damaged headcover for Callaway Rogue ST",
     "The headcover for my Callaway Rogue ST driver arrived damaged. Can I get a replacement? The headcover has a noticeable tear on the side, which I found right out of the box. While it may seem like a minor issue, I want to make sure that all parts of my equipment are in top condition, especially since I paid for premium quality. Could you please let me know how I can get a replacement headcover? Is there a form I need to fill out, or can this be handled directly through customer service? I appreciate your assistance in resolving this matter."),
    ("Odyssey Stroke Lab Putter grip replacement",
     "The grip on my Odyssey Stroke Lab Putter is wearing out. How can I order a replacement grip? I have really enjoyed using the Stroke Lab Putter, but the grip has started to wear out significantly, making it uncomfortable to use. I would prefer to replace it with the same model of grip that originally came with the putter. Could you provide instructions on how to order a replacement, or is there a recommended retailer that stocks these grips? Additionally, if there are any tips for replacing the grip myself, I would appreciate that information as well."),
    ("Jaws Full Toe wedge rusting prematurely",
     "My Jaws Full Toe wedge is rusting sooner than expected. Is this normal for the raw face finish? I understand that raw wedges are supposed to rust over time to provide better spin, but the amount of rust appearing on my wedge after only a few weeks seems excessive. I have been careful to dry it after each use, but it continues to rust rapidly. Could this be a defect, or is it typical for this model? I would appreciate any guidance you can provide on how to care for the wedge to ensure optimal performance without compromising its condition."),
    ("Warranty inquiry for Rogue ST MAX Fairway Wood",
     "The paint on my Rogue ST MAX Fairway Wood is chipping. Is this covered under warranty? I have used the club for only a few rounds, and I am already seeing significant paint chipping on the crown. I have been very careful with it, and I store it properly when not in use. I would like to know if this type of cosmetic damage is covered under warranty, and if so, what steps I need to take to have it repaired or replaced. It is important to me that my equipment not only performs well but also looks good, and I am disappointed to see this kind of wear so early on."),
    ("Custom fitting question for Big Bertha B21",
     "I want to get my Big Bertha B21 driver custom fitted. What options are available through Callaway? I have heard a lot about how custom fitting can significantly improve performance, and I am interested in exploring the options for my Big Bertha B21 driver. Could you provide information on where I can get professionally fitted? Does Callaway offer any fitting services directly, or should I visit an authorized retailer? Additionally, I would like to know what kind of adjustments can be made during the fitting process, such as loft, lie, and shaft selection. Any information on costs and scheduling would also be appreciated."),
    ("Replacement shaft for Epic Speed Driver",
     "The shaft on my Epic Speed Driver broke. Can I get a replacement through Callaway? I was hitting my driver at the range when the shaft suddenly snapped near the hosel. I am quite surprised, as I did not feel that I made an unusual swing, and I have always taken good care of my clubs. I would like to know if I can get a replacement shaft through Callaway and what the process would be. Should I send the entire club in for inspection, or is it possible to just replace the shaft locally with an authorized retailer? Any guidance you can provide would be very helpful."),
    ("Experiencing unexpected wear on Odyssey White Hot Putter grip",
     "The grip on my Odyssey White Hot Putter is wearing out much faster than I anticipated, even after minimal usage. Is it possible to get a replacement or is this a defect?"),
    ("Seeking advice on lost distance with Rogue ST MAX Driver",
     "I have noticed a significant drop in distance with my Rogue ST MAX Driver recently. I am looking for some advice on how to regain the lost yardage and improve my swing performance."),
    # Adding 100 more options for different Callaway product issues
    ("XR Speed Driver losing distance", 
     "I've noticed that my XR Speed Driver isn't giving me the same distance as it used to. Could this be due to wear on the clubface, or might it be an issue with my swing mechanics? Any suggestions for how to regain lost yardage would be appreciated."),
    ("Damaged grip on Epic Flash Star driver", 
     "The grip on my Epic Flash Star driver has torn, and I'm not sure how to replace it. Can you advise on whether I can get the same grip from Callaway or if there's a suitable alternative available?"),
    ("Cracks in the hosel of MAVRIK Max irons", 
     "I've found cracks in the hosel area of my MAVRIK Max irons after a few rounds. Is this normal wear, or should I file a warranty claim? I'm concerned about the long-term durability of the irons."),
    ("Alignment aid fading on Odyssey Triple Track putter", 
     "The alignment aid on my Odyssey Triple Track putter is fading, making it difficult to line up putts. Is there a way to restore or replace the alignment markings? I find the Triple Track feature very useful, so I'd like to keep it in good condition."),
    ("Customization options for Epic Speed Triple Diamond driver", 
     "I recently purchased the Epic Speed Triple Diamond driver and was wondering about the customization options. Can I get it adjusted to optimize for fade or draw bias? I would love to get professional fitting recommendations."),
    ("Weight adjustment on Big Bertha Fusion driver", 
     "I'd like to adjust the weight on my Big Bertha Fusion driver. Are there additional weight kits available through Callaway that could help me fine-tune my swing? I'm particularly interested in promoting a higher ball flight."),
    ("Damaged zipper on ORG 14 Cart Bag", 
     "The zipper on my ORG 14 Cart Bag has broken after about a month of use. I love the bag but need to know if it can be repaired or if there's a way to replace the entire bag under warranty."),
    ("Cracked face on Rogue Sub Zero fairway wood", 
     "The face of my Rogue Sub Zero fairway wood has developed a noticeable crack. I'm very concerned about continuing to use it, as it may break completely. Is this covered under warranty, and can I get a replacement club?"),
    ("Custom fitting for MAVRIK Max Hybrid", 
     "I recently bought a MAVRIK Max Hybrid and would like to get it custom fitted. What types of adjustments can be made, and is there a professional fitter in my area authorized by Callaway to help with this?"),
    ("Replacement strap for Chev Dry Stand Bag", 
     "One of the straps on my Chev Dry Stand Bag broke. How can I go about getting a replacement strap, and is this something that Callaway would cover under warranty? I want to make sure the replacement matches the original."),
    ("Issue with grip size on XR OS irons", 
     "The grips on my XR OS irons feel too small, and I am interested in upgrading to midsize grips. Does Callaway offer larger grips, and could this be done as a warranty service, or should I go to a local retailer?"),
    ("Face angle adjustment on Apex MB irons", 
     "Can the face angle be adjusted on my Apex MB irons? I feel like the irons are slightly closed at address, which is affecting my shot accuracy. I'd like to make an adjustment to ensure the irons are properly aligned."),
    ("Warranty question on cracked hosel of Big Bertha Hybrid", 
     "I have a Big Bertha Hybrid with a cracked hosel, and I'd like to know if this is covered under warranty. I have been very careful with the club, and it developed the crack during regular use. What are the steps for submitting a warranty claim?"),
    ("Epic Max Driver shaft upgrade options", 
     "I'm considering upgrading the shaft on my Epic Max Driver to suit my swing speed better. Are there specific shaft options available that are recommended by Callaway? I would like to ensure I get the best performance possible out of the driver."),
    ("Rogue X Irons distance consistency issue", 
     "I'm experiencing inconsistent distances with my Rogue X irons. Some shots are coming up short while others fly farther than expected. Could this be related to the clubhead, shaft, or my swing? Any recommendations for improving consistency would be appreciated."),
    ("Damaged headcover for MAVRIK Pro driver", 
     "The headcover for my MAVRIK Pro driver arrived with visible tears. Is it possible to get a replacement? I want to make sure all my equipment is well-protected, especially during transport."),
    ("Warranty claim for cracked face on XR Speed fairway wood", 
     "The face on my XR Speed fairway wood cracked after a recent round. I'm quite disappointed as it was purchased less than a year ago. How do I initiate a warranty claim, and will I receive a replacement club?"),
    ("Tear in strap of Hyper Lite Zero Stand Bag", 
     "The strap on my Hyper Lite Zero Stand Bag tore after only a few uses. I would like to know if I can get a replacement under warranty, and if not, what the best option is for repairing the strap."),
    ("Grip replacement for Jaws MD5 wedge", 
     "The grip on my Jaws MD5 wedge has worn out, and I'm not sure what replacement grip would be best. Can Callaway provide a similar grip to the original, and is there a recommended method for replacing it?"),
    ("Loft adjustment advice for Big Bertha Fusion irons", 
     "I am considering adjusting the loft on my Big Bertha Fusion irons to improve my trajectory. What are the recommended loft settings, and does Callaway provide guidance for these types of adjustments?"),
    ("Paint chipping on Epic Flash Sub Zero driver", 
     "The paint on my Epic Flash Sub Zero driver has started chipping after just a few rounds. Is this covered under warranty, and if so, what do I need to do to submit a claim? I'd like to ensure that my equipment looks as good as it performs."),
    ("Weight screw missing on Apex Pro irons", 
     "I've noticed that one of the weight screws on my Apex Pro irons is missing. Is it possible to order a replacement screw from Callaway, and could this affect my performance on the course? I'd appreciate any advice on fixing this issue."),
    ("Poor sound feedback on Rogue ST MAX driver", 
     "The sound feedback on my Rogue ST MAX driver seems very muted, unlike what I experienced when I initially purchased it. Could this be a defect in the club, or might something have happened to the internal structure? I'd like to know how to diagnose and address this problem."),
    ("Adjustable hosel issue on MAVRIK Sub Zero fairway wood", 
     "The adjustable hosel on my MAVRIK Sub Zero fairway wood is loose, and I'm worried it might impact my shots. Is this a common issue, and how can it be fixed? Should I bring it in for repair, or is there a home remedy I can try?"),
    ("MAVRIK Max irons losing chrome finish", 
     "I've noticed that the chrome finish on my MAVRIK Max irons is wearing off, especially on the face. I am careful with my clubs, and I find it disappointing that this is happening. Is this covered under warranty, and can anything be done to restore the finish?"),
    ("XR Speed Hybrid distance control issues", 
     "I am struggling to control my distances with the XR Speed Hybrid. Sometimes I hit it long, and other times it comes up short. Could this be related to the shaft or the head design? I need some help understanding what adjustments could improve consistency.")
])

# Populate lists
for _ in range(num_rows):
    subject, body = random.choice(email_content)
    subject_list.append(subject)
    body_list.append(body)
    email_from_list.append(f"{fake.user_name()}@{fake.free_email_domain()}")

# Create the DataFrame
data = {
    "email_id": [str(uuid.uuid4()) for _ in range(num_rows)],
    "email_from": email_from_list,
    "email_to": ["support@callawaygolf.com"] * num_rows,
    "email_cc": [None] * num_rows,
    "email_bcc": [None] * num_rows,
    "subject": subject_list,
    "email_body": body_list,
    "received_time": [datetime.now() - timedelta(days=random.randint(0, 365)) for _ in range(num_rows)],
    "email_status": [random.choice(["Open", "In Progress", "Closed"]) for _ in range(num_rows)],
    "priority": [random.choice(["High", "Medium", "Low"]) for _ in range(num_rows)],
    "response_due_by": [datetime.now() + timedelta(days=random.randint(0, 5)) for _ in range(num_rows)],
    "assigned_to": [f"agent{random.randint(1, 10)}" for _ in range(num_rows)],
    "tags": [["Product Issue", "Golf Club", "Golf Ball"]] * num_rows,
    "attachments": [["attachment1.pdf", "attachment2.jpg"]] * num_rows,
    "cortex_sentiment": [None] * num_rows,
    "cortex_sentiment_normalized": [None] * num_rows,
    "cortex_summarize": [None] * num_rows,
    "cortex_classify_text": [None] * num_rows,
    "cortex_extract_answer": [None] * num_rows,
    "cortex_translate": [None] * num_rows,
    "cortex_detected_language": [None] * num_rows,
    "case_number": [f"007{str(i).zfill(5)}" for i in range(1, num_rows + 1)]
}

# Create DataFrame
df = pd.DataFrame(data)

# Convert to Snowpark DataFrame
snowpark_df = session.create_dataframe(df)

# Insert the data into the customer_support_emails table
snowpark_df.write.save_as_table("customer_support_emails", mode="append")

st.write(f"Inserted {num_rows} random rows into customer_support_emails table.")

In [ ]:
'''# Import Python packages
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import udf
from snowflake.snowpark.types import StringType
import uuid

# Snowflake session
session = get_active_session()

# Retrieve existing entries from the customer_support_emails table
customer_support_emails_df = session.table("customer_support_emails")

# Generate unique case numbers starting with '006' and having 8 digits
def generate_case_number():
    return f"006{str(uuid.uuid4().int % 100000).zfill(5)}"

# Register the UDF to generate unique case numbers
generate_case_number_udf = udf(generate_case_number, return_type=StringType())

# Add a unique case number to each entry in the DataFrame
updated_df = customer_support_emails_df.with_column(
    "case_number",
    generate_case_number_udf()
)

# Update the customer_support_emails table with the new case_number column
updated_df.write.save_as_table("customer_support_emails", mode="overwrite")

print("Updated all entries in the customer_support_emails table to include unique case numbers.")


In [ ]:
'''import re
from snowflake.snowpark.context import get_active_session

# Snowflake session
session = get_active_session()


# Function to determine the category based on regex matches
def categorize_email(email_body):
    email_body_upper = email_body.upper()
    
    driver_matches = len(re.findall(r'\bDRIVER\b|\bDRIVERS\b|\bDRIVING\b|\bTEE OFF\b|\bOFF THE TEE\b|\bBIG BERTHA\b|\bPARADYM\b|\bROGUE ST\b|\bEPIC\b|\bMAVRIK\b', email_body_upper))
    iron_matches = len(re.findall(r'\bIRON\b|\bIRONS\b|[0-9]+\sIRON|[0-9]+\sIRONS|\bAPEX\b|\bROGUE\b|\bMAVRIK IRON\b|\bPARADYM IRON\b', email_body_upper))
    wedge_matches = len(re.findall(r'\bWEDGE\b|\bWEDGES\b|\bSAND WEDGE\b|\bPITCHING WEDGE\b|\bGAP WEDGE\b|\bLOB WEDGE\b|\bJAWS\b|\bMD[0-9]\b|\bMAC DADDY\b', email_body_upper))
    putter_matches = len(re.findall(r'\bPUTTER\b|\bPUTTERS\b|\bPUTTING\b|\bPUTT\b|\bODYSSEY\b|\bWHITE HOT\b|\bSTROKE LAB\b|\bTEN\b|\bTRIPLE TRACK\b|\b2-BALL\b', email_body_upper))
    wood_matches = len(re.findall(r'\bFAIRWAY WOOD\b|\bWOOD\b|\bWOODS\b|\b3 WOOD\b|\b5 WOOD\b|\b7 WOOD\b|\bPARADYM WOOD\b|\bROGUE WOOD\b|\bMAVRIK WOOD\b', email_body_upper))
    ball_matches = len(re.findall(r'\bBALL\b|\bBALLS\b|\bGOLF BALL\b|\bGOLF BALLS\b|\bCHROME SOFT\b|\bSUPERSOFT\b|\bWARBIRD\b|\bERC\b|\bREVA\b', email_body_upper))
    bag_matches = len(re.findall(r'\bBAG\b|\bBAGS\b|\bGOLF BAG\b|\bCARRY\b|\bCART BAG\b|\bSTAND BAG\b|\bHYPER DRY\b|\bORG\b|\bFAIRWAY\b', email_body_upper))
    accessory_matches = len(re.findall(r'\bGLOVE\b|\bTEE\b|\bMARKER\b|\bTOWEL\b|\bACCESSORY\b|\bACCESSORIES\b|\bHEADCOVER\b|\bHEAD COVER\b|\bGRIP\b|\bGRIPS\b', email_body_upper))
    
    matches = {
        'Drivers': driver_matches,
        'Irons': iron_matches,
        'Wedges': wedge_matches,
        'Putters': putter_matches,
        'Fairway Woods': wood_matches,
        'Golf Balls': ball_matches,
        'Bags': bag_matches,
        'Accessories': accessory_matches
    }
    
    max_matches = max(matches.values())
    if max_matches == 0:
        return 'Unclassified'
    
    for category, count in matches.items():
        if count == max_matches:
            return category

# Fetch unclassified emails from the database
emails = session.sql("""
    SELECT EMAIL_ID, EMAIL_BODY
    FROM CUSTOMER_SUPPORT_EMAILS
    WHERE CORTEX_CLASSIFY_TEXT IS NULL 
       OR CORTEX_CLASSIFY_TEXT = ''
       OR LEFT(CORTEX_CLASSIFY_TEXT, 1) = '{'
       OR LEFT(CORTEX_CLASSIFY_TEXT, 1) = '['
""").collect()

# Update each email with its category
for row in emails:
    email_id = row['EMAIL_ID']
    email_body = row['EMAIL_BODY']
    category = categorize_email(email_body)
    session.sql(
        """
        UPDATE CUSTOMER_SUPPORT_EMAILS
        SET CORTEX_CLASSIFY_TEXT = '%s'
        WHERE EMAIL_ID = '%s'
        """ % (category, email_id)
    ).collect()


In [ ]:
--TRUNCATE customer_support_emails;